In [2]:
#import libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


In [3]:
#초모수 값 지정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.random.set_seed(787)

In [4]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################


In [8]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
print(len(test_images),len(train_labels))
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)    
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(len(test_images))

10000 60000


In [16]:
#모델 형성
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.keras.activations.softmax)) #tf.nn.relu))  ##왜 relu
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
       
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

In [17]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

In [12]:
#for checkpoint
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "trainving_11_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_dataset, epochs =15,
          validation_data =test_dataset,
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/15
599/600 [============================>.] - ETA: 0s - loss: 3.6449 - accuracy: 0.1462
Epoch 00001: saving model to trainving_11_1/cp-0001.ckpt
600/600 [==============================] - 169s 282ms/step - loss: 3.6427 - accuracy: 0.1461 - val_loss: 2.3060 - val_accuracy: 0.1135
Epoch 2/15
599/600 [============================>.] - ETA: 0s - loss: 3.7971 - accuracy: 0.1164
Epoch 00002: saving model to trainving_11_1/cp-0002.ckpt
600/600 [==============================] - 173s 288ms/step - loss: 3.7974 - accuracy: 0.1164 - val_loss: 3.2332 - val_accuracy: 0.2028
Epoch 3/15
100/600 [====>.........................] - ETA: 2:30 - loss: 3.7439 - accuracy: 0.1376

KeyboardInterrupt: 

In [18]:
#for checkpoint
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "trainving_11_1_softmax/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_dataset, epochs =5,
          validation_data =test_dataset,
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/5
599/600 [============================>.] - ETA: 0s - loss: 9.6294 - accuracy: 0.0977
Epoch 00001: saving model to trainving_11_1_softmax/cp-0001.ckpt
600/600 [==============================] - 183s 306ms/step - loss: 9.6305 - accuracy: 0.0977 - val_loss: 11.3085 - val_accuracy: 0.0982
Epoch 2/5
599/600 [============================>.] - ETA: 0s - loss: 9.9973 - accuracy: 0.0974
Epoch 00002: saving model to trainving_11_1_softmax/cp-0002.ckpt
600/600 [==============================] - 162s 271ms/step - loss: 9.9956 - accuracy: 0.0974 - val_loss: 11.3085 - val_accuracy: 0.0982
Epoch 3/5
599/600 [============================>.] - ETA: 0s - loss: 10.0037 - accuracy: 0.0974
Epoch 00003: saving model to trainving_11_1_softmax/cp-0003.ckpt
600/600 [==============================] - 150s 250ms/step - loss: 10.0026 - accuracy: 0.0974 - val_loss: 11.3085 - val_accuracy: 0.0982
Epoch 4/5
599/600 [============================>.] - ETA: 0s - loss: 10.0357 - accuracy: 0.0974
Epoch 00004: s